In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
from seminartools.models.four_quarter_average_model import FourQuarterAverageModel
from seminartools.data import read_merged

# 1. Read data 

In [4]:
# expanding window crossval
from seminartools.time_series_split import ExpandingWindowSplit

tss = ExpandingWindowSplit(start_date = "2000-01-01")
tss

In [5]:
df = read_merged(coreInf=True).reset_index()
df['country'].unique()

c:\users\vandi\onedrive\documenten\github\seminar\seminartools\data.py:173: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  df = df.resample("Q").apply(lambda x: (1 + x).prod() - 1)  # quarterly return
c:\users\vandi\onedrive\documenten\github\seminar\seminartools\data.py:409: FutureWarning: 'Q' is deprecated and will be removed in a future version, please use 'QE' instead.
  df = df.resample("Q").mean()  # Mean interest rate


array(['United States', 'Japan', 'Canada', 'Switzerland', 'France',
       'Italy', 'Portugal', 'Sweden', 'United Kingdom', 'Denmark',
       'Germany', 'Netherlands'], dtype=object)

# 2. get summary stats for all models

In [6]:
from seminartools.models.utils import get_stats
from seminartools.models.four_quarter_average_model import FourQuarterAverageModel
from seminartools.models.pca_var_model import PCAVARModel
from seminartools.models.armax_model import ARMAXModel
from seminartools.models.uc_sv_ss_model import UCSVSSModel
from seminartools.models.random_effects_model import RandomEffectsModel
from seminartools.models.distance_model import DistanceModel
from seminartools.models.mucsvss_model import MUCSVSSModel

muc_model = MUCSVSSModel(num_particles=100008, stochastic_seasonality=True)
muc_model.load_from_disk("../../models/mucsvss_coreinf_model_intra_continent_100008_stochastic_all.pkl")
muc_model

In [7]:
exog_columns = [
    "gdp_growth",
    "interest_rate",
    "unemployment_rate",
    "commodity_CRUDE_PETRO",
    "commodity_iNATGAS",
    "commodity_iAGRICULTURE",
    "commodity_iMETMIN",
    "commodity_iPRECIOUSMET",
]

stats = get_stats(
    [
        #("pca_var_2", lambda: PCAVARModel(num_pcs = 2)),
        #("pca_var_3", lambda: PCAVARModel()),
        #("pca_var_4", lambda: PCAVARModel(num_pcs = 4)),
        #("pca_var_5", lambda: PCAVARModel(num_pcs = 5)),
        #("pca_var_6", lambda: PCAVARModel(num_pcs = 6)),
         ("4qa", lambda: FourQuarterAverageModel()),
         ("pca_var", lambda: PCAVARModel()),
         ("ar", lambda: ARMAXModel(max_p=2, max_q=0)),
         (
             "arx",
             lambda: ARMAXModel(max_p=2, max_q=0, exogenous_columns=exog_columns),
         ),
         ("arma", lambda: ARMAXModel(max_p=2, max_q=2)),
         (
             "armax",
             lambda: ARMAXModel(max_p=2, max_q=2, exogenous_columns=exog_columns),
         ),
        (
             "uc_sv",
             lambda: UCSVSSModel(num_particles=10000, stochastic_seasonality=False),
         ),
         (
             "uc_sv_ss",
             lambda: UCSVSSModel(num_particles=10000, stochastic_seasonality=True),
         ),
         ("random_effects", lambda: RandomEffectsModel()),
         (
             "random_effects_x",
             lambda: RandomEffectsModel(exogenous_columns=exog_columns),
         ),
         ("distance", lambda: DistanceModel()),
         ("distance_x", lambda: DistanceModel(exogenous_columns=exog_columns)),
         ("mucsvss", lambda: muc_model),
    ],
    df,
    tss,
    num_cores_parallel_models=1,  # only 1 model at once because models use joblib internally and nested joblib gives issues
)
stats

c:\Users\vandi\AppData\Local\anaconda3\envs\py311\Lib\site-packages\statsmodels\base\model.py:1914: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(
c:\Users\vandi\AppData\Local\anaconda3\envs\py311\Lib\site-packages\statsmodels\base\model.py:1914: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(
c:\Users\vandi\AppData\Local\anaconda3\envs\py311\Lib\site-packages\statsmodels\base\model.py:1914: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message w

Fitting model on the full dataset... (model=<seminartools.models.uc_sv_ss_model.UCSVSSModel object at 0x00000280CCF92310>)


100%|██████████| 12/12 [00:00<00:00, 22.03it/s]


Fitted!


c:\Users\vandi\AppData\Local\anaconda3\envs\py311\Lib\site-packages\statsmodels\base\model.py:1914: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(


...

Fitted!


c:\Users\vandi\AppData\Local\anaconda3\envs\py311\Lib\site-packages\statsmodels\base\model.py:1914: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(
c:\Users\vandi\AppData\Local\anaconda3\envs\py311\Lib\site-packages\statsmodels\base\model.py:1914: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(
c:\Users\vandi\AppData\Local\anaconda3\envs\py311\Lib\site-packages\statsmodels\base\model.py:1914: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message w

Fitting model on the full dataset... (model=<seminartools.models.mucsvss_model.MUCSVSSModel object at 0x00000280CA216C10>)
Fitted!


c:\Users\vandi\AppData\Local\anaconda3\envs\py311\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
c:\Users\vandi\AppData\Local\anaconda3\envs\py311\Lib\site-packages\statsmodels\base\model.py:1914: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(


,mse,mae,r2,mz_intercept,mz_slope,mz_r2,mz_wald_test_statistic,p-value
uc_sv_ss,0.000016,0.002813,0.514789,0.000491,0.960248,0.519328,4.637153,0.00989830103256279
arma,0.000027,0.003806,0.215259,0.001269,0.653647,0.299335,51.117483,1.0816789345301043e-21
4qa,0.000028,0.003922,0.173718,0.000835,0.831466,0.182588,5.458379,0.00438758963658192
pca_var,0.000029,0.004109,0.138627,0.002249,0.696019,0.258225,81.422429,1.6436412339662267e-33
ar,0.000029,0.004031,0.146727,0.001045,0.658756,0.205244,31.365998,7.179245368302443e-14
uc_sv,0.000033,0.004270,0.005305,0.001966,0.525602,0.036789,16.049033,1.3851055539488037e-07
random_effects,0.000054,0.005256,-0.609459,0.005928,-0.406460,0.051976,339.430733,1.4884338557965465e-112
random_effects_x,0.000057,0.005375,-0.682713,0.004639,-0.230072,0.024365,352.584739,6.82765197532687e-116
distance,0.000057,0.005429,-0.699563,0.005808,-0.392976,0.057846,391.103515,2.244681732510366e-125
arx,0.000063,0.005925,-0.833421,0.003116,0.193604,0.050391,396.482863,1.9250750392520505e-122


In [12]:
# format r2 column to 2 
format_stats = stats.copy()
format_stats['p-value'] =format_stats['p-value'].astype(float)
round_cols = ["r2", "mz_intercept", "mz_slope", "mz_r2", "mz_wald_test_statistic", "p-value"]
for col in round_cols:
    format_stats[col] = format_stats[col].apply(lambda x: round(x, 4))

format_stats['mz_wald_test_statistic'] = format_stats['mz_wald_test_statistic'].astype(str) + '(' + format_stats['p-value'].astype(str) + ')'
format_stats.drop(columns = ['p-value'], inplace = True)

#get relative MSPE
format_stats.loc[:,["mse","mae"]] = format_stats.loc[:,["mse","mae"]].div(format_stats.loc["uc_sv",["mse","mae"]])
format_stats

C:\Users\vandi\AppData\Local\Temp\ipykernel_32692\2427537230.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.4877991667770533 0.8079193287352654 0.8265524921791184
 0.8642208845439384 0.8784758052769396 1.0 1.621287197000918
 1.6950798871345256 1.712053075151907 1.8875733101207488
 2.0429926418633397 2.8172836148720553 53.62764879697612]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  format_stats.loc[:,["mse","mae"]] = format_stats.loc[:,["mse","mae"]].div(format_stats.loc["uc_sv",["mse","mae"]])
C:\Users\vandi\AppData\Local\Temp\ipykernel_32692\2427537230.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.6587714763837939 0.8915017646660011 0.918583359142991
 0.9624100987313788 0.9440168738296869 1.0 1.2310178595431147
 1.258838972683467 1.2715384757056358 1.38782230418647 1.39285831740

,mse,mae,r2,mz_intercept,mz_slope,mz_r2,mz_wald_test_statistic
uc_sv_ss,0.487799,0.658771,0.5148,0.0005,0.9602,0.5193,4.6372(0.0099)
arma,0.807919,0.891502,0.2153,0.0013,0.6536,0.2993,51.1175(0.0)
4qa,0.826552,0.918583,0.1737,0.0008,0.8315,0.1826,5.4584(0.0044)
pca_var,0.864221,0.96241,0.1386,0.0022,0.6960,0.2582,81.4224(0.0)
ar,0.878476,0.944017,0.1467,0.0010,0.6588,0.2052,31.366(0.0)
uc_sv,1.0,1.0,0.0053,0.0020,0.5256,0.0368,16.049(0.0)
random_effects,1.621287,1.231018,-0.6095,0.0059,-0.4065,0.0520,339.4307(0.0)
random_effects_x,1.69508,1.258839,-0.6827,0.0046,-0.2301,0.0244,352.5847(0.0)
distance,1.712053,1.271538,-0.6996,0.0058,-0.3930,0.0578,391.1035(0.0)
arx,1.887573,1.387822,-0.8334,0.0031,0.1936,0.0504,396.4829(0.0)


In [15]:
# generate latex code for the stats table
print(
    # replace _ by \_ in columns and index
    format_stats.pipe(lambda df: df.rename(columns=lambda x: x.replace("_", "\\_")))
    .pipe(lambda df: df.rename(index=lambda x: x.replace("_", "\\_")))
    .to_latex(
        index=True,
        caption="MSPE, MAE, $R^2$, and Mincer-Zarnowitz statistics, core inflation, out-of-sample, 2000-2023",
        label="tab:model_eval_core_inf",
        float_format="%.4f",
    )
)

\begin{table}
\caption{MSPE, MAE, $R^2$, and Mincer-Zarnowitz statistics, core inflation, out-of-sample, 2000-2023}
\label{tab:model_eval_core_inf}
\begin{tabular}{lllrrrrl}
\toprule
 & mse & mae & r2 & mz\_intercept & mz\_slope & mz\_r2 & mz\_wald\_test\_statistic \\
\midrule
uc\_sv\_ss & 0.4878 & 0.6588 & 0.5148 & 0.0005 & 0.9602 & 0.5193 & 4.6372(0.0099) \\
arma & 0.8079 & 0.8915 & 0.2153 & 0.0013 & 0.6536 & 0.2993 & 51.1175(0.0) \\
4qa & 0.8266 & 0.9186 & 0.1737 & 0.0008 & 0.8315 & 0.1826 & 5.4584(0.0044) \\
pca\_var & 0.8642 & 0.9624 & 0.1386 & 0.0022 & 0.6960 & 0.2582 & 81.4224(0.0) \\
ar & 0.8785 & 0.9440 & 0.1467 & 0.0010 & 0.6588 & 0.2052 & 31.366(0.0) \\
uc\_sv & 1.0000 & 1.0000 & 0.0053 & 0.0020 & 0.5256 & 0.0368 & 16.049(0.0) \\
random\_effects & 1.6213 & 1.2310 & -0.6095 & 0.0059 & -0.4065 & 0.0520 & 339.4307(0.0) \\
random\_effects\_x & 1.6951 & 1.2588 & -0.6827 & 0.0046 & -0.2301 & 0.0244 & 352.5847(0.0) \\
distance & 1.7121 & 1.2715 & -0.6996 & 0.0058 & -0.3930 & 0.0578

# 3. table HCPI using the 12 countries

In [9]:
df['country'].unique()

array(['United States', 'Japan', 'Canada', 'Switzerland', 'France',
       'Italy', 'Portugal', 'Sweden', 'United Kingdom', 'Denmark',
       'Germany', 'Netherlands'], dtype=object)

In [10]:
countries = ['United States', 'Japan', 'Canada', 'Switzerland', 'France',
       'Italy', 'Portugal', 'Sweden', 'United Kingdom', 'Denmark',
       'Germany', 'Netherlands']

In [11]:
dfs = read_merged(only_countries= countries).reset_index()
dfs

,index,country,date,inflation,gdp_growth,interest_rate,unemployment_rate,commodity_CRUDE_PETRO,commodity_iNATGAS,commodity_iAGRICULTURE,commodity_iMETMIN,commodity_iPRECIOUSMET
0,0,Japan,1977-01-01,0.022951,2.187257,6.333333,2.3,0.00000,0.052425,0.144757,0.041940,0.112740
1,1,Canada,1977-01-01,0.023904,1.533830,8.166667,8.8,0.00000,0.052425,0.144757,0.041940,0.112740
2,2,United States,1977-01-01,0.017231,1.186233,4.700000,8.2,0.00000,0.052425,0.144757,0.041940,0.112740
3,3,Japan,1977-04-01,0.027244,0.703788,5.000000,2.0,-0.00400,0.043586,-0.066688,-0.045412,-0.060029
4,4,Canada,1977-04-01,0.023346,0.510302,7.666667,8.0,-0.00400,0.043586,-0.066688,-0.045412,-0.060029
...,...,...,...,...,...,...,...,...,...,...,...,...
1317,2164,Netherlands,2023-01-01,-0.013667,-0.494664,3.000000,3.7,-0.02041,-0.585458,0.002037,0.007418,0.043183
1318,2165,Switzerland,2023-01-01,0.010084,0.309543,1.166667,4.4,-0.02041,-0.585458,0.002037,0.007418,0.043183
1319,2166,Italy,2023-01-01,0.003938,0.552944,3.000000,8.3,-0.02041,-0.585458,0.002037,0.007418,0.043183
1320,2167,Canada,2023-01-01,0.006080,0.610026,4.500000,5.4,-0.02041,-0.585458,0.002037,0.007418,0.043183


In [12]:

exog_columns = [
    "gdp_growth",
    "interest_rate",
    "unemployment_rate",
    "commodity_CRUDE_PETRO",
    "commodity_iNATGAS",
    "commodity_iAGRICULTURE",
    "commodity_iMETMIN",
    "commodity_iPRECIOUSMET",
]

stats = get_stats(
    [
        ("4qa", lambda: FourQuarterAverageModel()),
        ("ar", lambda: ARMAXModel(max_p=2, max_q=0)),
        (
            "arx",
            lambda: ARMAXModel(max_p=2, max_q=0, exogenous_columns=exog_columns),
        ),
        ("arma", lambda: ARMAXModel(max_p=2, max_q=2)),
        (
            "armax",
            lambda: ARMAXModel(max_p=2, max_q=2, exogenous_columns=exog_columns),
        ),
        (
            "uc_sv",
            lambda: UCSVSSModel(num_particles=10000, stochastic_seasonality=False),
        ),
        (
            "uc_sv_ss",
            lambda: UCSVSSModel(num_particles=10000, stochastic_seasonality=True),
        ),
        ("random_effects", lambda: RandomEffectsModel()),
        (
            "random_effects_x",
            lambda: RandomEffectsModel(exogenous_columns=exog_columns),
        ),
        ("distance", lambda: DistanceModel()),
        ("distance_x", lambda: DistanceModel(exogenous_columns=exog_columns)),
    ],
    dfs,
    tss,
    num_cores_parallel_models=1,  # only 1 model at once because models use joblib internally and nested joblib gives issues
)
stats

/home/maurits/miniconda3/envs/develop/lib/python3.11/site-packages/statsmodels/base/model.py:1906: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(
/home/maurits/miniconda3/envs/develop/lib/python3.11/site-packages/statsmodels/base/model.py:1906: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(
/home/maurits/private/seminar/seminartools/models/armax_model.py:211: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use 

Fitting model on the full dataset... (model=<seminartools.models.uc_sv_ss_model.UCSVSSModel object at 0x7efbee88ddd0>)


100%|██████████| 12/12 [00:00<00:00, 29.39it/s]


Fitted!


/home/maurits/miniconda3/envs/develop/lib/python3.11/site-packages/statsmodels/base/model.py:1906: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(


...

/home/maurits/miniconda3/envs/develop/lib/python3.11/site-packages/statsmodels/base/model.py:1906: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(


/home/maurits/miniconda3/envs/develop/lib/python3.11/site-packages/statsmodels/base/model.py:1906: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(
/home/maurits/miniconda3/envs/develop/lib/python3.11/site-packages/pycountry/db.py:51: UserWarning: Country's official_name not found. Country name provided instead.
  warnings.warn(warning_message, UserWarning)
/home/maurits/miniconda3/envs/develop/lib/python3.11/site-packages/pycountry/db.py:51: UserWarning: Country's common_name not found. Country name provided instead.
  warnings.warn(warning_message, UserWarning)
/home/maurits/miniconda3/envs/develop/lib/python3.11/site-packages/pycountry/db.py:51: UserWarning: Country's official_name not found. Country name provided instead.
  warnings.warn(warning_message, UserWarning)
/home/mau

/home/maurits/miniconda3/envs/develop/lib/python3.11/site-packages/statsmodels/base/model.py:1906: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(
Creating regression coefficients: 100%|██████████| 9/9 [00:02<00:00,  3.80it/s]


/home/maurits/miniconda3/envs/develop/lib/python3.11/site-packages/statsmodels/base/model.py:1906: FutureWarning: The behavior of wald_test will change after 0.14 to returning scalar test statistic values. To get the future behavior now, set scalar to True. To silence this message while retaining the legacy behavior, set scalar to False.
  warnings.warn(


,mse,mae,r2,mz_intercept,mz_slope,mz_r2,mz_wald_test_statistic,p-value
uc_sv_ss,0.000033,0.003971,0.318768,0.001059,0.814733,0.337382,13.792430,1.237689071716356e-06
4qa,0.000042,0.004714,0.125978,0.001671,0.671936,0.166367,24.370110,4.6194904290365215e-11
uc_sv,0.000047,0.004977,0.029068,0.002137,0.568927,0.070518,21.896154,4.972290200914971e-10
ar,0.000052,0.005220,-0.001561,0.002097,0.501266,0.063979,29.828206,3.014255382768282e-13
random_effects,0.000060,0.005601,-0.241196,0.005846,-0.223648,0.007983,122.200818,4.5010470920131025e-48
random_effects_x,0.000061,0.005577,-0.260044,0.004536,0.030753,0.000243,126.660062,1.2915797098649395e-49
distance,0.000062,0.005657,-0.274499,0.005589,-0.181915,0.006526,137.616627,2.3389188145146147e-53
distance_x,0.000078,0.006440,-0.609887,0.004706,-0.018445,0.000184,296.854253,2.2653576378238196e-101
arx,0.000181,0.008683,-2.458343,0.004481,0.020182,0.001028,1048.769823,1.786331214115679e-230
arma,0.000276,0.006805,-4.291627,0.004273,0.058948,0.016906,1866.997323,3.94982513137e-312


In [13]:
# format r2 column to 2 
format_stats = stats.copy()
format_stats['p-value'] =format_stats['p-value'].astype(float)
round_cols = ["r2", "mz_intercept", "mz_slope", "mz_r2", "mz_wald_test_statistic", "p-value"]
for col in round_cols:
    format_stats[col] = format_stats[col].apply(lambda x: round(x, 3))

format_stats['mz_wald_test_statistic'] = format_stats['mz_wald_test_statistic'].astype(str) + '(' + format_stats['p-value'].astype(str) + ')'
format_stats.drop(columns = ['p-value'], inplace = True)

# get relative to UCSV
format_stats.loc[:,["mse","mae"]] = format_stats.loc[:,["mse","mae"]].div(format_stats.loc["uc_sv",["mse","mae"]])
format_stats

,mse,mae,r2,mz_intercept,mz_slope,mz_r2,mz_wald_test_statistic
uc_sv_ss,0.000033,0.003971,0.319,0.001,0.815,0.337,13.792(0.0)
4qa,0.000042,0.004714,0.126,0.002,0.672,0.166,24.37(0.0)
uc_sv,0.000047,0.004977,0.029,0.002,0.569,0.071,21.896(0.0)
ar,0.000052,0.005220,-0.002,0.002,0.501,0.064,29.828(0.0)
random_effects,0.000060,0.005601,-0.241,0.006,-0.224,0.008,122.201(0.0)
random_effects_x,0.000061,0.005577,-0.260,0.005,0.031,0.000,126.66(0.0)
distance,0.000062,0.005657,-0.274,0.006,-0.182,0.007,137.617(0.0)
distance_x,0.000078,0.006440,-0.610,0.005,-0.018,0.000,296.854(0.0)
arx,0.000181,0.008683,-2.458,0.004,0.020,0.001,1048.77(0.0)
arma,0.000276,0.006805,-4.292,0.004,0.059,0.017,1866.997(0.0)


In [14]:
# generate latex code for the stats table
print(
    # replace _ by \_ in columns and index
    format_stats.pipe(lambda df: df.rename(columns=lambda x: x.replace("_", "\\_")))
    .pipe(lambda df: df.rename(index=lambda x: x.replace("_", "\\_")))
    .to_latex(
        index=True,
        caption="MSPE, MAE, $R^2$, and Mincer-Zarnowitz statistics, out-of-sample, 2000-2023",
        label="tab:model_eval",
        float_format="%.4f",
    )
)

\begin{table}
\caption{MSPE, MAE, $R^2$, and Mincer-Zarnowitz statistics, out-of-sample, 2000-2023}
\label{tab:model_eval}
\begin{tabular}{lrrrrrrl}
\toprule
 & mse & mae & r2 & mz\_intercept & mz\_slope & mz\_r2 & mz\_wald\_test\_statistic \\
\midrule
uc\_sv\_ss & 0.0000 & 0.0040 & 0.3190 & 0.0010 & 0.8150 & 0.3370 & 13.792(0.0) \\
4qa & 0.0000 & 0.0047 & 0.1260 & 0.0020 & 0.6720 & 0.1660 & 24.37(0.0) \\
uc\_sv & 0.0000 & 0.0050 & 0.0290 & 0.0020 & 0.5690 & 0.0710 & 21.896(0.0) \\
ar & 0.0001 & 0.0052 & -0.0020 & 0.0020 & 0.5010 & 0.0640 & 29.828(0.0) \\
random\_effects & 0.0001 & 0.0056 & -0.2410 & 0.0060 & -0.2240 & 0.0080 & 122.201(0.0) \\
random\_effects\_x & 0.0001 & 0.0056 & -0.2600 & 0.0050 & 0.0310 & 0.0000 & 126.66(0.0) \\
distance & 0.0001 & 0.0057 & -0.2740 & 0.0060 & -0.1820 & 0.0070 & 137.617(0.0) \\
distance\_x & 0.0001 & 0.0064 & -0.6100 & 0.0050 & -0.0180 & 0.0000 & 296.854(0.0) \\
arx & 0.0002 & 0.0087 & -2.4580 & 0.0040 & 0.0200 & 0.0010 & 1048.77(0.0) \\
arma & 0.00